In [3]:
import pandas as pd

# Path to your phenotype file on Google Drive
phenotype_file = 'C:/Users/GREESHMA/Desktop/cropformer/dataset/Ncii_2015_5locs_hybrids.txt'

try:
    # Read the file
    pheno_df = pd.read_csv(phenotype_file, sep='\s+')
    
    # Get the unique values from the 'loc' column
    unique_locations = pheno_df['loc'].unique()
    
    print("✅ Found the following unique location IDs:")
    for loc_id in unique_locations:
        print(f"- {loc_id}")

except FileNotFoundError:
    print(f"❌ Error: Could not find the phenotype file at '{phenotype_file}'. Please check the path.")

✅ Found the following unique location IDs:
- BJ
- HeB
- HN
- JL
- LN


In [8]:
import pandas as pd
from geopy.geocoders import Nominatim
import time

# --- Configuration ---
# A dictionary mapping your location IDs to their full names for searching
location_map = {
    'BJ': 'Beijing, China',
    'HeB': 'Hebei, China',
    'HN': 'Henan, China',
    'JL': 'Jilin, China',
    'LN': 'Liaoning, China'
}
output_file = 'location/location_coordinates.csv'
# ---------------------

print("🚀 Starting to geocode locations...")

# Initialize the geocoder (Nominatim uses OpenStreetMap data)
# A unique user_agent is required by their terms of service
geolocator = Nominatim(user_agent="crop-former-project-greeshma")

results_list = []

for loc_id, search_query in location_map.items():
    print(f"Searching for: '{search_query}'...")
    try:
        location = geolocator.geocode(search_query, timeout=10)
        if location:
            results_list.append({
                'Location ID': loc_id,
                'Full Name': search_query,
                'Latitude': location.latitude,
                'Longitude': location.longitude
            })
            print(f"  -> Found: Lat {location.latitude:.4f}, Lon {location.longitude:.4f}")
        else:
            results_list.append({
                'Location ID': loc_id,
                'Full Name': search_query,
                'Latitude': 'Not Found',
                'Longitude': 'Not Found'
            })
            print(f"  -> Could not find coordinates for '{search_query}'.")
        
        # Be respectful to the free API and wait a second between requests
        time.sleep(1)

    except Exception as e:
        print(f"An error occurred while searching for '{search_query}': {e}")

# Convert the results to a pandas DataFrame for a nice display
results_df = pd.DataFrame(results_list)

print("\n--- Geocoding Complete ---")
print(results_df)

# Save the results to a CSV file for later use
results_df.to_csv(output_file, index=False)
print(f"\n✅ Results have been saved to '{output_file}'")

🚀 Starting to geocode locations...
Searching for: 'Beijing, China'...
  -> Found: Lat 40.1906, Lon 116.4121
Searching for: 'Hebei, China'...
  -> Found: Lat 39.0000, Lon 116.0000
Searching for: 'Henan, China'...
  -> Found: Lat 34.0000, Lon 114.0000
Searching for: 'Jilin, China'...
  -> Found: Lat 43.5826, Lon 126.1266
Searching for: 'Liaoning, China'...
  -> Found: Lat 41.2374, Lon 122.9955

--- Geocoding Complete ---
  Location ID        Full Name   Latitude   Longitude
0          BJ   Beijing, China  40.190632  116.412144
1         HeB     Hebei, China  39.000000  116.000000
2          HN     Henan, China  34.000000  114.000000
3          JL     Jilin, China  43.582583  126.126618
4          LN  Liaoning, China  41.237411  122.995547

✅ Results have been saved to 'location/location_coordinates.csv'


In [5]:
import pandas as pd

# --- Configuration ---
coordinates_file = 'location/location_coordinates.csv'

# Corrected rural coordinates for Beijing (Shunyi District)
beijing_new_lat = 40.13
beijing_new_lon = 116.66
# ---------------------

try:
    # Read the existing coordinates file
    df = pd.read_csv(coordinates_file)
    
    print("--- Original Coordinates ---")
    print(df)
    
    # Find the row where 'Location ID' is 'BJ' and update its Latitude and Longitude
    df.loc[df['Location ID'] == 'BJ', ['Latitude', 'Longitude']] = [beijing_new_lat, beijing_new_lon]
    
    # Save the modified DataFrame back to the same file, overwriting it
    df.to_csv(coordinates_file, index=False)
    
    print("\n--- Modified Coordinates ---")
    print(df)
    
    print(f"\n✅ Successfully updated '{coordinates_file}' with the new coordinates for Beijing.")

except FileNotFoundError:
    print(f"❌ Error: The file '{coordinates_file}' was not found.")
except KeyError:
    print("❌ Error: Could not find the column 'Location ID' in the file. Please check the column name.")

--- Original Coordinates ---
  Location ID        Full Name   Latitude   Longitude
0          BJ   Beijing, China  40.190632  116.412144
1         HeB     Hebei, China  39.000000  116.000000
2          HN     Henan, China  34.000000  114.000000
3          JL     Jilin, China  43.728967  126.199737
4          LN  Liaoning, China  41.237411  122.995547

--- Modified Coordinates ---
  Location ID        Full Name   Latitude   Longitude
0          BJ   Beijing, China  40.130000  116.660000
1         HeB     Hebei, China  39.000000  116.000000
2          HN     Henan, China  34.000000  114.000000
3          JL     Jilin, China  43.728967  126.199737
4          LN  Liaoning, China  41.237411  122.995547

✅ Successfully updated 'location/location_coordinates.csv' with the new coordinates for Beijing.


In [2]:
import netCDF4
print(netCDF4.__version__)


1.7.2


In [1]:
import pandas as pd
import requests
import time
from tqdm import tqdm
import numpy as np
import os

# --- Configuration ---
coordinates_file = 'location/location_coordinates.csv'
output_file = 'location/location_weather.csv'
YEAR = 2015
START = f"{YEAR}0501"  # May 1st, 2015
END   = f"{YEAR}0930"  # Sep 30th, 2015
# ----------------------

print("🚀 Starting to fetch historical weather (Temp & Rain) from NASA POWER...")

try:
    locations_df = pd.read_csv(coordinates_file)
    weather_results = []

    for index, row in tqdm(locations_df.iterrows(), total=len(locations_df), desc="Fetching locations"):
        loc_id = row['Location ID']
        lat = row['Latitude']
        lon = row['Longitude']

        # --- MODIFIED: Added PRECTOTCORR to parameters ---
        url = (
            f"https://power.larc.nasa.gov/api/temporal/daily/point"
            f"?parameters=T2M,PRECTOTCORR"  # Requesting Temp and Precipitation
            f"&start={START}&end={END}"
            f"&latitude={lat}&longitude={lon}"
            f"&community=AG"
            f"&format=JSON"
        )

        response = requests.get(url)
        if response.status_code == 200:
            data = response.json()

            if "properties" in data and "parameter" in data["properties"]:
                params = data["properties"]["parameter"]
                
                # Calculate average temperature
                temps = [temp for temp in params.get("T2M", {}).values() if temp > -990]
                avg_temp = np.mean(temps) if temps else None

                # Calculate total precipitation
                rains = [rain for rain in params.get("PRECTOTCORR", {}).values() if rain > -990]
                total_rain = np.sum(rains) if rains else None
                
                weather_results.append({
                    "Location_ID": loc_id,
                    "Avg_Season_Temp_C": avg_temp,
                    "Total_Rain_mm": total_rain
                })
                print(f"\n  -> {loc_id}: Avg Temp = {avg_temp:.2f}°C, Total Rain = {total_rain:.2f} mm")
            else:
                print(f"\nWarning: No data returned for {loc_id}")
        else:
            print(f"\nWarning: Failed to fetch data for {loc_id}. Status: {response.status_code}")

        # avoid hitting request limits
        time.sleep(0.5)

    # Save results
    weather_df = pd.DataFrame(weather_results)
    if not os.path.exists('location'):
        os.makedirs('location')
    weather_df.to_csv(output_file, index=False)

    print("\n--- Weather Data Fetching Complete ---")
    print(weather_df.round(2))
    print(f"\n✅ Results have been saved to '{output_file}'")

except Exception as e:
    import traceback
    print(f"\n❌ An unexpected error occurred: {e}")
    traceback.print_exc()

🚀 Starting to fetch historical weather (Temp & Rain) from NASA POWER...


Fetching locations:   0%|                                                                        | 0/5 [00:00<?, ?it/s]


  -> BJ: Avg Temp = 22.97°C, Total Rain = 429.97 mm


Fetching locations:  20%|████████████▊                                                   | 1/5 [00:02<00:10,  2.69s/it]


  -> HeB: Avg Temp = 24.33°C, Total Rain = 418.85 mm


...


  -> JL: Avg Temp = 18.45°C, Total Rain = 453.84 mm


Fetching locations:  80%|███████████████████████████████████████████████████▏            | 4/5 [00:09<00:02,  2.43s/it]


  -> LN: Avg Temp = 20.52°C, Total Rain = 525.80 mm


Fetching locations: 100%|████████████████████████████████████████████████████████████████| 5/5 [00:11<00:00,  2.39s/it]


--- Weather Data Fetching Complete ---
  Location_ID  Avg_Season_Temp_C  Total_Rain_mm
0          BJ              22.97         429.97
1         HeB              24.33         418.85
2          HN              24.89         455.20
3          JL              18.45         453.84
4          LN              20.52         525.80

✅ Results have been saved to 'location/location_weather.csv'


In [13]:
import pandas as pd
import requests
from tqdm import tqdm
import numpy as np
import time
import os
import traceback

# ========================
# CONFIGURATION
# ========================
SOILGRID_URL = "https://rest.isric.org/soilgrids/v2.0/properties/query"
coordinates_file = 'location/location_coordinates.csv'
output_file = 'location/location_soil_data.csv'
# ------------------------

print("🚀 Starting to fetch soil data from SoilGrids...")

def fetch_soil(lat, lon):
    """Fetches soil pH and organic carbon from SoilGrids API with robust parsing."""
    params = {
        "lon": lon, "lat": lat, "property": ["phh2o", "ocd"],
        "depth": ["0-5cm", "5-15cm"], "value": ["mean"]
    }
    try:
        r = requests.get(SOILGRID_URL, params=params, timeout=30)
        if r.status_code != 200:
            return None, None
        
        data = r.json()
        ph_mean, ocd_mean = None, None
        
        layers = data.get("properties", {}).get("layers", [])
        for layer in layers:
            name = layer.get("name")
            depth_values = [
                d['values']['mean'] for d in layer.get('depths', [])
                if 'values' in d and 'mean' in d['values'] and d['values']['mean'] is not None
            ]
            if not depth_values: continue
            avg_value = np.mean(depth_values)
            if name == "phh2o": ph_mean = avg_value / 10.0
            elif name == "ocd": ocd_mean = avg_value
        
        return ph_mean, ocd_mean
    except Exception:
        return None, None

# --- Main Logic ---
try:
    # 1. Read coordinates from the file
    print(f"Reading coordinates from '{coordinates_file}'...")
    locations_df = pd.read_csv(coordinates_file)
    
    # 2. Apply the fix for Beijing's coordinates in the DataFrame
    print("Applying coordinate fix for Beijing (BJ)...")
    locations_df.loc[locations_df['Location ID'] == 'BJ', ['Latitude', 'Longitude']] = [40.13, 116.66] # Shunyi District

    # 3. Loop through the DataFrame and fetch data
    soil_results = []
    for index, row in tqdm(locations_df.iterrows(), total=len(locations_df), desc="Fetching locations"):
        loc_id = row['Location ID']
        lat = row['Latitude']
        lon = row['Longitude']
        
        soil_ph, soil_oc = fetch_soil(lat, lon)
        
        soil_results.append({
            "Location_ID": loc_id,
            "Soil_pH": round(soil_ph, 2) if soil_ph is not None else np.nan,
            "Soil_OrgCarbon": round(soil_oc, 2) if soil_oc is not None else np.nan
        })
        time.sleep(1)

    soil_df = pd.DataFrame(soil_results)

    # 4. Final safety check: If BJ is still missing, use HeB data
    bj_row = soil_df[soil_df['Location_ID'] == 'BJ']
    if bj_row.isnull().values.any():
        print("\nBeijing (BJ) data is missing after API call. Using data from nearby Hebei (HeB) as a substitute.")
        heb_data = soil_df[soil_df['Location_ID'] == 'HeB'].iloc[0]
        soil_df.loc[soil_df['Location_ID'] == 'BJ', ['Soil_pH', 'Soil_OrgCarbon']] = [heb_data['Soil_pH'], heb_data['Soil_OrgCarbon']]

    # 5. Save the final, complete DataFrame
    if not os.path.exists('location'):
        os.makedirs('location')
    soil_df.to_csv(output_file, index=False)

    print("\n--- Soil Data Fetching Complete ---")
    print(soil_df)
    print(f"\n✅ Results have been saved to '{output_file}'")

except FileNotFoundError:
    print(f"❌ Error: Could not find the coordinates file at '{coordinates_file}'.")
except Exception as e:
    print(f"\n❌ An unexpected error occurred: {e}")
    traceback.print_exc()

🚀 Starting to fetch soil data from SoilGrids...
Reading coordinates from 'location/location_coordinates.csv'...
Applying coordinate fix for Beijing (BJ)...


Fetching locations: 100%|████████████████████████████████████████████████████████████████| 5/5 [00:12<00:00,  2.47s/it]


Beijing (BJ) data is missing after API call. Using data from nearby Hebei (HeB) as a substitute.

--- Soil Data Fetching Complete ---
  Location_ID  Soil_pH  Soil_OrgCarbon
0          BJ     8.05           177.0
1         HeB     8.05           177.0
2          HN     7.75           222.0
3          JL     6.55           286.5
4          LN     6.75           218.0

✅ Results have been saved to 'location/location_soil_data.csv'


In [9]:
import pandas as pd
import os
import traceback

# --- Configuration ---
geno_file = 'csv_files/chr10_top10k_snps.csv'
weather_file = 'location/location_weather.csv'
soil_file = 'location/location_soil_data.csv'
phenotype_map_file = 'dataset/Ncii_2015_5locs_hybrids.txt'
output_file = 'csv_files/chr10_top10k_snps_with_env.csv'
# ---------------------

print("🚀 Starting to merge data using a robust method...")

try:
    # 1. Load all data files
    print("Loading data files...")
    geno_df = pd.read_csv(geno_file, index_col=0)
    weather_df = pd.read_csv(weather_file)
    soil_df = pd.read_csv(soil_file)
    pheno_map_df = pd.read_csv(phenotype_map_file, sep='\\s+')

    # 2. Create helper maps (dictionaries) for fast lookups
    # Merge weather and soil and set location as the index
    env_df = pd.merge(weather_df, soil_df, on='Location_ID').set_index('Location_ID')
    temp_map = env_df['Avg_Season_Temp_C'].to_dict()
    rain_map = env_df['Total_Rain_mm'].to_dict()  # <-- ADDED
    ph_map = env_df['Soil_pH'].to_dict()
    oc_map = env_df['Soil_OrgCarbon'].to_dict()
    
    # Create a map from Sample ID ('Lineid') to Location ID ('loc')
    sample_to_loc_map = pheno_map_df.drop_duplicates(subset='Lineid').set_index('Lineid')['loc']
    
    # 3. Use the .map() method to add new columns without losing the index
    print("Adding environmental data columns...")
    
    # First, find the location for each sample in our main dataframe
    loc_col = geno_df.index.map(sample_to_loc_map)
    
    # Now, use that location to find the correct environmental data from our maps
    geno_df['Avg_Temp_C'] = loc_col.map(temp_map)
    geno_df['Total_Rain_mm'] = loc_col.map(rain_map) # <-- ADDED
    geno_df['Soil_pH'] = loc_col.map(ph_map)
    geno_df['Soil_OrgCarbon'] = loc_col.map(oc_map)
    
    # 4. Save the result. The original index is preserved.
    geno_df.to_csv(output_file, index=True)

    print("\n--- Merge Complete ---")
    print(f"Final enriched data has {geno_df.shape[0]} samples and {geno_df.shape[1]} columns.")
    print(f"\n✅ Successfully saved to '{output_file}'")

except FileNotFoundError as e:
    print(f"❌ Error: A required file was not found. Please check this path: {e.filename}")
except Exception as e:
    print(f"\n❌ An unexpected error occurred: {e}")
    traceback.print_exc()

🚀 Starting to merge data using a robust method...
Loading data files...
Adding environmental data columns...

--- Merge Complete ---
Final enriched data has 6210 samples and 10004 columns.

✅ Successfully saved to 'csv_files/chr10_top10k_snps_with_env.csv'


In [10]:
import pandas as pd

# --- Configuration (using the paths from your error message) ---
geno_file = 'csv_files/chr10_top10k_snps_with_env.csv'
pheno_file = 'dataset/Ncii_2015_5locs_hybrids.txt'
# ---------------------

try:
    print("--- Checking Sample IDs from Genotype File ---")
    geno_df = pd.read_csv(geno_file, index_col=0)
    print("Index name:", geno_df.index.name)
    print("First 5 sample IDs:")
    print(geno_df.index[:5].tolist())
    
    print("\n" + "="*50 + "\n")
    
    print("--- Checking Sample IDs from Phenotype File ---")
    pheno_df = pd.read_csv(pheno_file, sep='\\s+')
    print("Column name: 'Lineid'")
    print("First 5 unique sample IDs:")
    print(pheno_df['Lineid'].unique()[:5].tolist())

except Exception as e:
    print(f"An error occurred: {e}")

--- Checking Sample IDs from Genotype File ---
Index name: IID
First 5 sample IDs:
['MG_115_X_MG_1528', 'MG_991_X_MG_1524', 'MG_162_X_MG_1540', 'MG_204_X_MG_1520', 'MG_68_X_MG_1545']


--- Checking Sample IDs from Phenotype File ---
Column name: 'Lineid'
First 5 unique sample IDs:
['MG_298_X_MG_1530', 'MG_298_X_MG_1534', 'MG_298_X_MG_1538', 'MG_298_X_MG_1540', 'MG_298_X_MG_1518']


In [11]:
import pandas as pd

geno_file = 'csv_files/chr10_top10k_snps_with_env.csv'

print("🔍 Reading genotype file...")

# Try reading first few rows
geno_df = pd.read_csv(geno_file)

print("\n--- File Overview ---")
print(f"Shape: {geno_df.shape}")
print("Columns:", geno_df.columns.tolist()[:10], "..." if geno_df.shape[1] > 10 else "")

# Check first few rows to see how the data looks
print("\n--- First 5 Rows ---")
print(geno_df.head())

# Check if any column name looks like it holds sample identifiers
print("\n--- Column name check ---")
possible_id_cols = [col for col in geno_df.columns if 'id' in col.lower() or 'sample' in col.lower() or 'line' in col.lower()]
print("Possible sample ID columns:", possible_id_cols)

# If first column looks like IDs (non-numeric), show a sample
first_col = geno_df.columns[0]
if not pd.api.types.is_numeric_dtype(geno_df[first_col]):
    print(f"\n✅ First column '{first_col}' might contain sample IDs:")
    print(geno_df[first_col].head())
else:
    print(f"\n⚠️ First column '{first_col}' seems numeric — sample IDs might be missing.")


🔍 Reading genotype file...

--- File Overview ---
Shape: (6210, 10005)
Columns: ['IID', 'chr10.s_623678', 'chr10.s_623766', 'chr10.s_623848', 'chr10.s_623933', 'chr10.s_623948', 'chr10.s_623982', 'chr10.s_624155', 'chr10.s_624413', 'chr10.s_624647'] ...

--- First 5 Rows ---
                IID  chr10.s_623678  chr10.s_623766  chr10.s_623848  \
0  MG_115_X_MG_1528               5               3               3   
1  MG_991_X_MG_1524               5               3               3   
2  MG_162_X_MG_1540               5               3               3   
3  MG_204_X_MG_1520               5               3               3   
4   MG_68_X_MG_1545               7               9               9   

   chr10.s_623933  chr10.s_623948  chr10.s_623982  chr10.s_624155  \
0               3               3               3               6   
1               3               3               3               6   
2               3               3               3               6   
3               3    

In [12]:
import pandas as pd
from sklearn.model_selection import train_test_split
import traceback

# --- IMPORTANT: CHOOSE YOUR TRAIT ---
# Make sure this is the same trait you used for feature selection.
TARGET_TRAIT = 'DTT'
# ------------------------------------

# --- Configuration ---
feature_selected_geno_file = 'csv_files/chr10_top10k_snps_with_env.csv'
phenotype_file = 'dataset/Ncii_2015_5locs_hybrids.txt'
# ---------------------

print("🚀 Starting Phase 2: Splitting data into training and testing sets...")

try:
    # 1. Load the data
    print(f"Loading feature-selected genotype data from '{feature_selected_geno_file}'...")
    X = pd.read_csv(feature_selected_geno_file, index_col=0)

    print(f"Loading and averaging phenotype data from '{phenotype_file}'...")
    pheno_df = pd.read_csv(phenotype_file, sep='\\s+')
    
    # --- THIS IS THE NEW, CORRECTED PART ---
    # Group by the sample ID and calculate the mean for the target trait
    y_series = pheno_df.groupby('Lineid')[TARGET_TRAIT].mean()
    # ---------------------------------------

    # 2. Align the datasets
    print("Aligning final genotype and phenotype data...")
    common_samples = X.index.intersection(y_series.index)
    X = X.loc[common_samples]
    y = y_series.loc[common_samples]
    
    # Drop any remaining missing values (e.g., if mean results in NaN)
    valid_indices = y.dropna().index
    y = y.loc[valid_indices]
    X = X.loc[valid_indices]

    print(f"Final dataset contains {len(X)} samples.")

    # 3. Split the data (80% train, 20% test)
    print("Splitting data (80% training, 20% testing)...")
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42
    )
    
    print(f"Training set size: {len(X_train)} samples")
    print(f"Testing set size: {len(X_test)} samples")

    # 4. Save the four new files
    print("Saving output files...")
    X_train.to_csv('X_train_environ.csv')
    y_train.to_csv('y_train_environ.csv', header=True)
    X_test.to_csv('X_test_environ.csv')
    y_test.to_csv('y_test_environ.csv', header=True)
    
    print("✅ Success! Data splitting is complete.")

except Exception as e:
    print(f"❌ An unexpected error occurred: {e}")
    traceback.print_exc()

🚀 Starting Phase 2: Splitting data into training and testing sets...
Loading feature-selected genotype data from 'csv_files/chr10_top10k_snps_with_env.csv'...
Loading and averaging phenotype data from 'dataset/Ncii_2015_5locs_hybrids.txt'...
Aligning final genotype and phenotype data...
Final dataset contains 5831 samples.
Splitting data (80% training, 20% testing)...
Training set size: 4664 samples
Testing set size: 1167 samples
Saving output files...
✅ Success! Data splitting is complete.
